<h1>Лабораторная работа №3. Подготовка обучающей и тестовой выборки, кросс-валидация и подбор гиперпараметров на примере метода ближайших соседей.<h1>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from warnings import simplefilter

simplefilter('ignore')

In [2]:
df = pd.read_csv('top50.csv', encoding='ISO-8859-1')
df.head()

,Unnamed: 0,Track.Name,Artist.Name,Genre,Beats.Per.Minute,Energy,Danceability,Loudness..dB..,Liveness,Valence.,Length.,Acousticness..,Speechiness.,Popularity
0,1,Señorita,Shawn Mendes,canadian pop,117,55,76,-6,8,75,191,4,3,79
1,2,China,Anuel AA,reggaeton flow,105,81,79,-4,8,61,302,8,9,92
2,3,boyfriend (with Social House),Ariana Grande,dance pop,190,80,40,-4,16,70,186,12,46,85
3,4,Beautiful People (feat. Khalid),Ed Sheeran,pop,93,65,64,-8,8,55,198,12,19,86
4,5,Goodbyes (Feat. Young Thug),Post Malone,dfw rap,150,65,58,-4,11,18,175,45,7,94


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        50 non-null     int64 
 1   Track.Name        50 non-null     object
 2   Artist.Name       50 non-null     object
 3   Genre             50 non-null     object
 4   Beats.Per.Minute  50 non-null     int64 
 5   Energy            50 non-null     int64 
 6   Danceability      50 non-null     int64 
 7   Loudness..dB..    50 non-null     int64 
 8   Liveness          50 non-null     int64 
 9   Valence.          50 non-null     int64 
 10  Length.           50 non-null     int64 
 11  Acousticness..    50 non-null     int64 
 12  Speechiness.      50 non-null     int64 
 13  Popularity        50 non-null     int64 
dtypes: int64(11), object(3)
memory usage: 5.6+ KB


Подготовка датасета

In [4]:
remove_cols = ['Unnamed: 0', 'Track.Name']
df.drop(remove_cols, axis=1, inplace=True)
df.head()

,Artist.Name,Genre,Beats.Per.Minute,Energy,Danceability,Loudness..dB..,Liveness,Valence.,Length.,Acousticness..,Speechiness.,Popularity
0,Shawn Mendes,canadian pop,117,55,76,-6,8,75,191,4,3,79
1,Anuel AA,reggaeton flow,105,81,79,-4,8,61,302,8,9,92
2,Ariana Grande,dance pop,190,80,40,-4,16,70,186,12,46,85
3,Ed Sheeran,pop,93,65,64,-8,8,55,198,12,19,86
4,Post Malone,dfw rap,150,65,58,-4,11,18,175,45,7,94


In [5]:
# Количество уникальных значений
category_cols = ['Artist.Name', 'Genre']
for c in category_cols:
    print(f'{c}: {df[c].unique().size}/{df[c].shape[0]}')

Artist.Name: 38/50
Genre: 21/50


In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le = LabelEncoder()
for c in category_cols:
    df[c] = le.fit_transform(df[[c]])
df.head()

,Artist.Name,Genre,Beats.Per.Minute,Energy,Danceability,Loudness..dB..,Liveness,Valence.,Length.,Acousticness..,Speechiness.,Popularity
0,32,6,117,55,76,-6,8,75,191,4,3,79
1,1,19,105,81,79,-4,8,61,302,8,9,92
2,2,8,190,80,40,-4,16,70,186,12,46,85
3,9,15,93,65,64,-8,8,55,198,12,19,86
4,28,9,150,65,58,-4,11,18,175,45,7,94


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Artist.Name       50 non-null     int64
 1   Genre             50 non-null     int64
 2   Beats.Per.Minute  50 non-null     int64
 3   Energy            50 non-null     int64
 4   Danceability      50 non-null     int64
 5   Loudness..dB..    50 non-null     int64
 6   Liveness          50 non-null     int64
 7   Valence.          50 non-null     int64
 8   Length.           50 non-null     int64
 9   Acousticness..    50 non-null     int64
 10  Speechiness.      50 non-null     int64
 11  Popularity        50 non-null     int64
dtypes: int64(12)
memory usage: 4.8 KB


In [8]:
df.dropna()

,Artist.Name,Genre,Beats.Per.Minute,Energy,Danceability,Loudness..dB..,Liveness,Valence.,Length.,Acousticness..,Speechiness.,Popularity
0,32,6,117,55,76,-6,8,75,191,4,3,79
1,1,19,105,81,79,-4,8,61,302,8,9,92
2,2,8,190,80,40,-4,16,70,186,12,46,85
3,9,15,93,65,64,-8,8,55,198,12,19,86
4,28,9,150,65,58,-4,11,18,175,45,7,94
5,9,15,102,68,80,-5,9,84,220,9,4,84
6,20,20,180,64,75,-6,7,23,131,2,29,92
7,30,15,111,68,48,-5,8,35,202,15,9,90
8,19,7,136,62,88,-6,11,64,157,5,10,87
9,4,11,135,43,70,-11,10,56,194,33,38,95


Видим, что в датасете нет пропусков. Выделим целевой признак.

In [9]:
target_col = 'Popularity'
y = df[target_col]
x = df.drop(target_col, axis=1)

Разделим выборку на тестовую и обучающую.

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3)

Обучение методом k ближайших соседей с произвольным k

In [11]:
def print_metric(y_test, y_pred):
    print(f"R^2_score: {r2_score(y_test, y_pred)}")
    print(f"MSE: {mean_squared_error(y_test, y_pred)}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred)}")

def print_cv_result(cv_model, x_test, y_test):
    print(f'Оптимизация метрики {cv_model.scoring}: {cv_model.best_score_}')
    print(f'Лучший параметр: {cv_model.best_params_}')
    print('Метрики на тестовом наборе')
    print_metric(y_test, cv_model.predict(x_test))
    print()

Возьмем k = 5

In [12]:
k = 5
regr_k = KNeighborsRegressor(n_neighbors=k)
regr_k.fit(x_train, y_train)
y_pred_base = regr_k.predict(x_test)
print_metric(y_test, y_pred_base)

R^2_score: 0.01166666666666738
MSE: 14.23199999999999
MAE: 2.8666666666666667


Кросс-валидация

In [13]:
metrics = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
cv_values = [5, 10]

for cv in cv_values:
    print('========== cv = {} =========='.format(cv))
    for metric in metrics:
        params = {'n_neighbors': range(1, 30)}
        knn_cv = GridSearchCV(KNeighborsRegressor(), params, cv=cv, scoring=metric, n_jobs=-1)
        knn_cv.fit(x_train, y_train)
        print_cv_result(knn_cv, x_test, y_test)

========== cv = 5 ==========
Оптимизация метрики r2: -0.09166418969860927
Лучший параметр: {'n_neighbors': 12}
Метрики на тестовом наборе
R^2_score: -0.17959104938271642
MSE: 16.986111111111118
MAE: 3.355555555555556

Оптимизация метрики neg_mean_squared_error: -20.255142857142857
Лучший параметр: {'n_neighbors': 10}
Метрики на тестовом наборе
R^2_score: -0.27518518518518476
MSE: 18.36266666666666
MAE: 3.4399999999999995

Оптимизация метрики neg_mean_absolute_error: -3.5417989417989424
Лучший параметр: {'n_neighbors': 27}
Метрики на тестовом наборе
R^2_score: -0.3805568256871408
MSE: 19.880018289894828
MAE: 3.5604938271604927

========== cv = 10 ==========
Оптимизация метрики r2: -2.82847213570181
Лучший параметр: {'n_neighbors': 5}
Метрики на тестовом наборе
R^2_score: 0.01166666666666738
MSE: 14.23199999999999
MAE: 2.8666666666666667

Оптимизация метрики neg_mean_squared_error: -18.14233333333333
Лучший параметр: {'n_neighbors': 5}
Метрики на тестовом наборе
R^2_score: 0.011666666666

In [14]:
best_k = 4
y_pred_best = KNeighborsRegressor(n_neighbors=best_k).fit(x_train, y_train).predict(x_test)

Сравним исходную и оптимальную модели

In [15]:
print('Basic model')
print_metric(y_test, y_pred_base)
print('==========================')
print('Optimal model')
print_metric(y_test, y_pred_best)

Basic model
R^2_score: 0.01166666666666738
MSE: 14.23199999999999
MAE: 2.8666666666666667
Optimal model
R^2_score: 0.06452546296296291
MSE: 13.470833333333333
MAE: 2.783333333333333
